In [1]:
import pandas as pd
import numpy as np

In [2]:
# -------------------------------------------------------------
# 0) 예시 데이터 
# -------------------------------------------------------------
data = pd.DataFrame({
    'Trip ID': [1, 2, 3, 4, 5],
    'Age': [25, 34, 45, 29, 40],                 # 제거할 컬럼
    'Car Color': ['Red', 'Blue', 'Green', 'Red', 'Black'],  # 제거할 컬럼
    'Destination From': ['A', 'B', 'C', 'D', 'E'],
    'Destination To': ['F', 'G', 'H', 'I', 'J'],
    'Distance (km)': [12.5, 25.3, 19.8, 30.2, 15.7]
})


In [3]:
data.head()

,Trip ID,Age,Car Color,Destination From,Destination To,Distance (km)
0,1,25,Red,A,F,12.5
1,2,34,Blue,B,G,25.3
2,3,45,Green,C,H,19.8
3,4,29,Red,D,I,30.2
4,5,40,Black,E,J,15.7


In [6]:
# -------------------------------------------------------------
# 1) 불필요한 컬럼(Age, Car Color) 제거
# -------------------------------------------------------------
data_cleaned = data.drop(columns = ['Age', 'Car Color'])

In [7]:
data_cleaned.head()

,Trip ID,Destination From,Destination To,Distance (km)
0,1,A,F,12.5
1,2,B,G,25.3
2,3,C,H,19.8
3,4,D,I,30.2
4,5,E,J,15.7


In [12]:
# -------------------------------------------------------------
# 2) 최대 이동 거리 탐색 및 해당 경로 필터링
# -------------------------------------------------------------
max_distance = data_cleaned['Distance (km)'].idxmax()
longest_trip = data_cleaned.loc[max_distance]

print("[결과] 가장 먼 거리의 출발지-도착지 경로")
print(longest_trip[['Destination From', 'Destination To', 'Distance (km)']])
print()

[결과] 가장 먼 거리의 출발지-도착지 경로
Destination From       D
Destination To         I
Distance (km)       30.2
Name: 3, dtype: object



In [13]:
# -------------------------------------------------------------
# 3) (전처리) 파생 변수 생성: 장거리 여부 플래그 (Feature Creation)
# 장거리 여부 표시
#    - 기준선을 하나 정해 두면(예: 20km) 집계·비율 확인이 쉬워짐
# is_long: 기준선 이상이면 1, 아니면 0 (type: int)
# -------------------------------------------------------------
THRESHOLD_KM = 20  # 목적 기준선
data_feat = data_cleaned.copy()
data_feat['is_long'] = (data_feat['Distance (km)'] >= THRESHOLD_KM).astype(int)

print("[파생변수] is_long (>=20km):")
print(data_feat[['Destination From', 'Destination To', 'Distance (km)', 'is_long']])
print()

[파생변수] is_long (>=20km):
  Destination From Destination To  Distance (km)  is_long
0                A              F           12.5        0
1                B              G           25.3        1
2                C              H           19.8        0
3                D              I           30.2        1
4                E              J           15.7        0



In [15]:
# -------------------------------------------------------------
# 4) (전처리) 결측치/이상치 점검
#    - 결측치 요약, IQR 기반 이상값 후보 탐지
# -------------------------------------------------------------
# 4-1) 결측치 점검 (컬럼 별 결측치 개수)
na_summary = data_feat.isna().sum()
print("[결측치 점검]")
print(na_summary)
print()

# 4-2) 이상치 점검 (IQR 방법) - 'Distance (km)'에 대한 이상치만 점검
q1 = data_feat['Distance (km)'].quantile(0.25)
q3 = data_feat['Distance (km)'].quantile(0.75)
iqr = q3 - q1
low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
outliers = data_feat[ (data_feat['Distance (km)'] <= low) | (data_feat['Distance (km)'] >= high) ]

print("[이상치(IQR) 점검]")
print(f"Q1={q1:.2f}, Q3={q3:.2f}, IQR={iqr:.2f}, 범위=({low:.2f} ~ {high:.2f})")
print("이상치 후보:\n", outliers if not outliers.empty else "없음")
print()

[결측치 점검]
Trip ID             0
Destination From    0
Destination To      0
Distance (km)       0
is_long             0
dtype: int64

[이상치(IQR) 점검]
Q1=15.70, Q3=25.30, IQR=9.60, 범위=(1.30 ~ 39.70)
이상치 후보:
 없음



In [16]:
# -------------------------------------------------------------
# 5) (전처리) 범주형 인코딩 (One-Hot Encoding)
#    - 문자형(출발지/도착지)을 0/1 숫자 형태 컬럼으로 변환 - get_dummies 활용
# -------------------------------------------------------------
encoded = pd.get_dummies(
    data_feat,
    columns=['Destination From', 'Destination To'],
    drop_first=False,
    dtype='int8'
)

print("[원-핫 인코딩 결과 컬럼]")
print(list(encoded.columns))
print()

[원-핫 인코딩 결과 컬럼]
['Trip ID', 'Distance (km)', 'is_long', 'Destination From_A', 'Destination From_B', 'Destination From_C', 'Destination From_D', 'Destination From_E', 'Destination To_F', 'Destination To_G', 'Destination To_H', 'Destination To_I', 'Destination To_J']



In [17]:
# -------------------------------------------------------------
# 6) 최종 전처리 산출물
#    - 불필요 컬럼 제거 + 파생변수 + 인코딩까지 마친 DataFrame 확인
# -------------------------------------------------------------
print("[최종 전처리 데이터(preview)]")
print(encoded.head())

[최종 전처리 데이터(preview)]
   Trip ID  Distance (km)  is_long  Destination From_A  Destination From_B  \
0        1           12.5        0                   1                   0   
1        2           25.3        1                   0                   1   
2        3           19.8        0                   0                   0   
3        4           30.2        1                   0                   0   
4        5           15.7        0                   0                   0   

   Destination From_C  Destination From_D  Destination From_E  \
0                   0                   0                   0   
1                   0                   0                   0   
2                   1                   0                   0   
3                   0                   1                   0   
4                   0                   0                   1   

   Destination To_F  Destination To_G  Destination To_H  Destination To_I  \
0                 1                 0    